# Exemplo 16: Meteorologia - Sistema de Alerta (Dask)

Este notebook demonstra o processamento de dados históricos de sensores meteorológicos para identificar padrões de ondas de calor usando **Dask**.

**Cenário**: Detectar dias onde a temperatura máxima excedeu a média histórica em mais de 5 graus.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Dados Meteorológicos

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('weather_data', exist_ok=True)

def generate_weather_data(year):
    stations = [f'STATION-{i:02d}' for i in range(10)]
    dates = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='h')
    num_records = len(dates)
    
    data = []
    for station in stations:
        temp_base = 25
        # Simular verão quente em 2024
        if year == 2024:
            temp_base = 30
            
        temps = np.random.normal(temp_base, 5, num_records)
        
        df = pd.DataFrame({
            'station_id': station,
            'timestamp': dates,
            'temperature': temps,
            'humidity': np.random.uniform(40, 90, num_records)
        })
        # Forçar onda de calor na Estação 05 em Jan
        if station == 'STATION-05':
            mask = df['timestamp'].dt.month == 1
            df.loc[mask, 'temperature'] += 10
            
        data.append(df)
        
    full_df = pd.concat(data)
    full_df.to_csv(f'weather_data/year_{year}.csv', index=False)

print("Gerando dados de 5 anos...")
for y in range(2020, 2025):
    generate_weather_data(y)

## 3. Análise de Ondas de Calor

In [ ]:
ddf = dd.read_csv('weather_data/year_*.csv')

# Converter timestamp
ddf['timestamp'] = dd.to_datetime(ddf['timestamp'])
ddf['date'] = ddf['timestamp'].dt.date

# Calcular temperatura máxima diária por estação
daily_max = ddf.groupby(['station_id', 'date'])['temperature'].max()

# Identificar alertas (> 40 graus)
alerts = daily_max[daily_max > 40]

print("Dias de Calor Extremo:")
print(alerts.compute().sort_values(ascending=False).head(20))

In [ ]:
client.close()